In [1]:

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
import pandas as pd
import sys
from pathlib import Path
from calibrationUtils import preprocess_genres
import os
import pickle





In [3]:
sys.path.append("/home/caio/dev/")

In [4]:

#from evaluation import Evaluate
from bprMf.bpr_mf import bprMf

from torch.utils.data import random_split, DataLoader
from torch.optim import Adam

from torch import device, cuda, tensor


In [5]:
dev = device('cuda' if cuda.is_available() else 'cpu')
dev

device(type='cuda')

# Dynamic calibration


In this notebook, we'll explore some ideas that model the evolution of user preferences in calibration. Namely, the work proposed by D.C da Silva et al (2025 ) - Considering Time and Feature Entropy in Calibrated Recommendations

## Read Data and Preprocess

In [6]:
path = "/home/caio/dev/dynamicTasteDistortion/data/movielens-1m/ml_1m.pkl"

df = pd.read_pickle(path)

In [7]:
df

,item,genres,user,rating,timestamp,binarized_rating
0,0,"[animation, children's, comedy]",0,5,978824268,1
1,0,"[animation, children's, comedy]",5,4,978237008,1
2,0,"[animation, children's, comedy]",7,4,978233496,1
3,0,"[animation, children's, comedy]",8,5,978225952,1
4,0,"[animation, children's, comedy]",9,5,978226474,1
...,...,...,...,...,...,...
1000204,3705,"[drama, thriller]",5811,4,992072099,1
1000205,3705,"[drama, thriller]",5830,3,986223125,0
1000206,3705,"[drama, thriller]",5836,4,1011902656,1
1000207,3705,"[drama, thriller]",5926,1,979852537,0


In [8]:
df["relevant"] = df["binarized_rating"]

In [9]:
df

,item,genres,user,rating,timestamp,binarized_rating,relevant
0,0,"[animation, children's, comedy]",0,5,978824268,1,1
1,0,"[animation, children's, comedy]",5,4,978237008,1,1
2,0,"[animation, children's, comedy]",7,4,978233496,1,1
3,0,"[animation, children's, comedy]",8,5,978225952,1,1
4,0,"[animation, children's, comedy]",9,5,978226474,1,1
...,...,...,...,...,...,...,...
1000204,3705,"[drama, thriller]",5811,4,992072099,1,1
1000205,3705,"[drama, thriller]",5830,3,986223125,0,0
1000206,3705,"[drama, thriller]",5836,4,1011902656,1,1
1000207,3705,"[drama, thriller]",5926,1,979852537,0,0


## Training a model

In [10]:
train_frac = 0.7

train_df = df.sample(frac=train_frac, random_state=42)

test_df = df.drop(train_df.index)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


n_users = df.user.max() + 1
n_items = df.item.max() + 1


In [11]:

model = bprMf(num_users=n_users, num_items=n_items, factors=30, reg_lambda=1e-4, n_epochs=1, dev=dev)

In [12]:
train_df.to_pickle("teste.pkl")

In [13]:
pd.read_pickle("teste.pkl")

,item,genres,user,rating,timestamp,binarized_rating,relevant
0,1288,"[action, horror]",1273,3,974813507,0,0
1,3383,"[action, thriller]",4574,4,964444259,1,1
2,969,[crime],3415,4,967411726,1,1
3,3072,[drama],228,3,1039501857,0,0
4,632,"[action, adventure, fantasy]",4799,2,962944712,0,0
...,...,...,...,...,...,...,...
687423,1117,[drama],3071,3,969940168,0,0
687424,2053,"[comedy, drama, romance]",623,5,975996117,1,1
687425,480,"[action, thriller]",5681,5,959043049,1,1
687426,1851,"[children's, comedy]",3575,3,966723171,0,0


In [14]:
train_df

,item,genres,user,rating,timestamp,binarized_rating,relevant
0,1288,"[action, horror]",1273,3,974813507,0,0
1,3383,"[action, thriller]",4574,4,964444259,1,1
2,969,[crime],3415,4,967411726,1,1
3,3072,[drama],228,3,1039501857,0,0
4,632,"[action, adventure, fantasy]",4799,2,962944712,0,0
...,...,...,...,...,...,...,...
687423,1117,[drama],3071,3,969940168,0,0
687424,2053,"[comedy, drama, romance]",623,5,975996117,1,1
687425,480,"[action, thriller]",5681,5,959043049,1,1
687426,1851,"[children's, comedy]",3575,3,966723171,0,0


In [15]:
losses = model.fit(train_df)

Epochs: 100%|██████████| 1/1 [00:16<00:00, 16.54s/it]


In [16]:
rec_df = model.score(df).explode(["top_k_rec_id", "top_k_rec_score"])

In [17]:
rec_df

,user,top_k_rec_id,top_k_rec_score
0,0,253,4.137317
0,0,2651,4.136602
0,0,1106,4.09663
0,0,579,4.050355
0,0,1848,4.027755
...,...,...,...
5288,5726,2131,2.977644
5288,5726,1545,2.956565
5288,5726,1492,2.952589
5288,5726,370,2.948869


## Calibrating

In [18]:
from calibration import Calibration
from calibrationIO import instantiate_calibrator

In [19]:
READ_LOCALLY = False

In [20]:
from calibrationUtils import build_tensors_from_df

In [21]:
df

,item,genres,user,rating,timestamp,binarized_rating,relevant
0,0,"[animation, children's, comedy]",0,5,978824268,1,1
1,0,"[animation, children's, comedy]",5,4,978237008,1,1
2,0,"[animation, children's, comedy]",7,4,978233496,1,1
3,0,"[animation, children's, comedy]",8,5,978225952,1,1
4,0,"[animation, children's, comedy]",9,5,978226474,1,1
...,...,...,...,...,...,...,...
1000204,3705,"[drama, thriller]",5811,4,992072099,1,1
1000205,3705,"[drama, thriller]",5830,3,986223125,0,0
1000206,3705,"[drama, thriller]",5836,4,1011902656,1,1
1000207,3705,"[drama, thriller]",5926,1,979852537,0,0


In [ ]:
lambdas = [0.1, 0.3, 0.5, 0.7, 0.9, 0.99, 1]

In [22]:
#calibrator_linear_time = instantiate_calibrator(df, 'linear_time', 'steck', rec_df, READ_LOCALLY=READ_LOCALLY, _lambda=0.1)

calibrator = Calibration(df, rec_df, 'linear_time', 'steck', _lambda=0.99)

In [23]:
import torch

In [24]:
from calibratedRecs.calibrationUtils import USER_COL, ITEM_COL

In [25]:
calibrator._mace(is_calibrated=False)

0.03370068594813347

In [26]:
calibrator.calibrate_for_users()

100%|██████████| 5289/5289 [19:58<00:00,  4.41it/s]


In [27]:
calibrator._mace(is_calibrated=True)

0.02178620733320713

### Investigação do erro do MACE

In [35]:
calibrator.user_history_tensor[0]

tensor([0.0289, 0.0235, 0.1256, 0.1331, 0.1256, 0.0120, 0.0000, 0.3093, 0.0180,
        0.0000, 0.0000, 0.1121, 0.0000, 0.0443, 0.0203, 0.0285, 0.0188, 0.0000],
       device='cuda:0')

In [28]:
calibrator_linear_time._mace()

NameError: name 'calibrator_linear_time' is not defined

## GLEB Based calibration


GLEB (Global Local Entropy Based) Calibration is a breakthrough approach proposed by D.C et Al (2025). It benefits items with a mixture of genres instead of penalizing them, which is something that Stecks method does indirectly. This approach works by calibrating using the proportion of genres in the users history as well as the proportion of genres in each item. 

In [ ]:
calibrator_linear_time_gleb = instantiate_calibrator(df, 'linear_time', 'gleb', rec_df)

100%|██████████| 6040/6040 [24:03<00:00,  4.19it/s]


In [ ]:
calibrator_linear_time_gleb.mace()

100%|██████████| 6040/6040 [00:18<00:00, 330.54it/s]


0.011589026786283185